# Part B

## Step 1 - Adding Features

#### Load Data

In [ ]:
import pandas as pd

hotels_file_path = "./hotels_data.csv"  
df = pd.read_csv(hotels_file_path)

df.head()

#### Add columns and load to new CSV

In [ ]:
#Ensure dates are in correct format
df['Snapshot Date'] = pd.to_datetime(df['Snapshot Date'])
df['Checkin Date'] = pd.to_datetime(df['Checkin Date'])

df['DayDiff'] = (df['Checkin Date'] - df['Snapshot Date']).dt.days
df['WeekDay'] = df['Checkin Date'].dt.day_name()
df['DiscountDiff'] = df['Original Price'] - df['Discount Price']
df['DiscountPerc'] = (df['DiscountDiff'] / df['Original Price']) * 100

print(df.head())


**New csv format**

| Column Name       | Description                                                                                         | Example Value                       |
|--------------------|-----------------------------------------------------------------------------------------------------|-------------------------------------|
| **Snapshot ID**    | Unique identifier for each snapshot of data                                                        | 1                                   |
| **Snapshot Date**  | The date when the snapshot was taken                                                               | 2015-07-17                          |
| **Checkin Date**   | The date of check-in for the hotel                                                                 | 2015-08-12                          |
| **Days**           | Duration of the stay in days                                                                       | 5                                   |
| **Original Price** | Price of the stay without any discount (in dollars)                                                | 1178                                |
| **Discount Price** | Price of the stay after applying the discount (in dollars)                                         | 1040                                |
| **Discount Code**  | Code representing the type of discount applied (values 1-4, with 1 indicating no discount possible) | 1                                   |
| **Available Rooms**| Number of rooms available at the specified check-in date                                           | 6                                   |
| **Hotel Name**     | Name of the hotel                                                                                  | Best Western Plus Seaport Inn Downtown |
| **Hotel Stars**    | Star rating of the hotel                                                                           | 3                                   |
| **DayDiff**        | Number of days between the Snapshot Date and Checkin Date                                          | 26                                  |
| **WeekDay**        | Day of the week corresponding to the Checkin Date                                                  | Wednesday                           |
| **DiscountDiff**   | Difference between the Original Price and Discount Price (in dollars)                              | 138                                 |
| **DiscountPerc**   | Percentage of discount applied, calculated as `(DiscountDiff / Original Price) * 100`             | 11.714770797962649                           |


#### Save To CSV

In [ ]:
changed_hotels_path = "./hotels_data_changed.csv"
df.to_csv(changed_hotels_path, index=False)

## Step 2 - Best Discount Code


**b. Classification Algorithms:** 
   - After creating this dataset, we will implement various classification algorithms in Python to predict the maximum discount code given the input parameters. 
   - Implement and evaluate the following five algorithms: Random Forest, Decision Tree, Naïve Bayes, XGBoost, and a simple Random classifier.
   - For each algorithm, you need to experiment with different parameter settings to find the optimal combination that yields the best performance. 
   - Explanation of how each chosen parameter affects the algorithm's performance. 

#### Data Preperation

In [ ]:
# Drop irrelevant columns

df = pd.read_csv(changed_hotels_path) 
df = df[['WeekDay', 'Snapshot Date', 'Checkin Date', 'DayDiff', 'Hotel Name', 'Discount Code']] 
df = df.rename(columns={'Discount Code': 'Class'})
df.head()

In [ ]:
# Feature Engineering

def map_days_to_numbers(df: pd.DataFrame) -> pd.DataFrame:
    weekday_mapping = {'Sunday': 1, 'Monday': 2, 'Tuesday': 3, 'Wednesday': 4, 'Thursday': 5, 'Friday': 6, 'Saturday': 7}
    df['WeekDay'] = df['WeekDay'].map(weekday_mapping)
    return df

def map_hotel_names_to_numbers(df: pd.DataFrame) -> pd.DataFrame:
    hotel_mapping = {hotel: i for i, hotel in enumerate(df['Hotel Name'].unique())}
    df['Hotel_Index'] = df['Hotel Name'].map(hotel_mapping)
    df.drop(['Hotel Name'], axis=1, inplace=True)
    return df

def map_date_to_numbers(df: pd.DataFrame, old_coloumn: str, new_coloumn: str) -> pd.DataFrame:
    df[old_coloumn] = pd.to_datetime(df[old_coloumn])
    df[f'{new_coloumn}_Year'] = df[old_coloumn].dt.year
    df[f'{new_coloumn}_Month'] = df[old_coloumn].dt.month
    df[f'{new_coloumn}_Day'] = df[old_coloumn].dt.day
    df.drop([old_coloumn], axis=1, inplace=True)

    return df

In [ ]:
df = map_days_to_numbers(df)
df = map_hotel_names_to_numbers(df)
df = map_date_to_numbers(df, 'Snapshot Date', 'Snapshot')
df = map_date_to_numbers(df, 'Checkin Date', 'Checkin')
df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

numerical_features = ['DayDiff', 'Snapshot_Year', 'Snapshot_Month', 'Snapshot_Day', 'Checkin_Year', 'Checkin_Month', 'Checkin_Day']
categorical_features = ['WeekDay', 'Hotel_Index']

# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(15, 8))
axes = axes.flatten()

# Plot histograms for numerical features
for i, feature in enumerate(numerical_features):
  sns.histplot(data=df, x=feature, ax=axes[i])
  axes[i].set_title(f'Distribution of {feature}')

# Plot bar charts for categorical features (assuming enough data for each category)
for i, feature in enumerate(categorical_features):
  sns.countplot(data=df, x=feature, ax=axes[i])
  axes[i].set_title(f'Distribution of {feature}')

plt.tight_layout()
plt.show()

In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = df[['WeekDay', 'DayDiff', 'Hotel_Index', 'Snapshot_Year', 'Snapshot_Month', 'Snapshot_Day', 'Checkin_Year', 'Checkin_Month', 'Checkin_Day']]
y = df['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

#### Naive Bayes
We have the following feature types:

* **Continuous:**
    - `DayDiff` 
    - `Snapshot_Year`
    - `Snapshot_Month`
    - `Snapshot_Day`
    - `Checkin_Year`
    - `Checkin_Month`
    - `Checkin_Day`

* **Categorical:**
    - `WeekDay` 
    - `Hotel_Index`

**Naive Bayes Variants and Suitability:**

* **GaussianNB:**
    - **Best suited for continuous features.** It assumes that the features are normally distributed (Gaussian). Since we have several continuous features GaussianNB seems like the choice.

* **MultinomialNB:**
    - **Suitable for discrete features.** While `WeekDay` and `Hotel_Index` can be treated as categorical, they don't inherently represent counts or frequencies.

* **BernoulliNB:**
    - **Suitable for binary features.** Not applicable in this case as none of the features are binary.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
y_pred = naive_bayes.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred)) 

## Step 3 - Clustering Based on Price Polices 


### Data Preparation  


Identify the 150 hotels with the most data in the dataset and extract their records.

In [ ]:
import pandas as pd

file_path = "./hotels_data_changed.csv"  
df = pd.read_csv(file_path)

hotel_counts = df['Hotel Name'].value_counts()
top_150_hotels = hotel_counts.head(150).index
filtered_df = df[df['Hotel Name'].isin(top_150_hotels)]

filtered_df.head()

Find the 40 most common check-in dates  in the dataset and extract their records.


In [ ]:
checkin_counts = filtered_df['Checkin Date'].value_counts()
top_40_checkin_dates = checkin_counts.head(40).index
filtered_checkin_df = filtered_df[filtered_df['Checkin Date'].isin(top_40_checkin_dates)]

filtered_checkin_df.head()

**Task**

Build a 160-dimensional feature vector for each hotel based on its discount pricing behavior. Each vector is constructed by:
- Filtering the top 150 hotels (by record count) and the top 40 checkin dates.
- For each hotel, extracting 4 discount prices (one per discount code) for each of the 40 checkin dates.
- If no data is available for a specific (checkin date, discount code) combination, mark it with `-1`.

**Plan**

1. **Group the Data:**  
   Group the filtered data by **Hotel Name**, **Checkin Date**, and **Discount Code**. For each group, compute the minimum discount price, ensuring that only the best (lowest) price per combination is selected.

2. **Pivot to Wide Format:**  
   Transform the grouped data into a wide format where:
   - Each row represents a single hotel.
   - Each column represents a unique (Checkin Date, Discount Code) combination, totaling 160 columns (40 dates × 4 codes).

3. **Fill Missing Data:**  
   - Reindex the pivoted DataFrame so that every hotel has all 160 combinations, filling missing entries with `-1`.




In [ ]:
# 1. Group by Hotel Name, Checkin Date, and Discount Code and select the minimum Discount Price.
grouped = (
    filtered_checkin_df
    .groupby(['Hotel Name', 'Checkin Date', 'Discount Code'])['Discount Price']
    .min()
    .reset_index()
)


# 2. Pivot the DataFrame so that:
#    - The index is 'Hotel Name'
#    - The columns are a MultiIndex with levels (Checkin Date, Discount Code)
#    - The values are the minimum discount prices.
pivot_df = grouped.pivot_table(index='Hotel Name',
                               columns=['Checkin Date', 'Discount Code'],
                               values='Discount Price')


# 3. Reindex the columns so that all 40 checkin dates and 4 discount codes are present.
#    Use the top_40_checkin_dates (from your earlier filtering) and the list [1, 2, 3, 4] for discount codes. 
all_combinations = pd.MultiIndex.from_product([top_40_checkin_dates, [1, 2, 3, 4]],
                                                names=['Checkin Date', 'Discount Code'])

pivot_df = pivot_df.reindex(columns=all_combinations, fill_value=-1)
pivot_df = pivot_df.fillna(-1)


pivot_df.columns = [
    col if isinstance(col, str) else f"{col[0]} - {col[1]}"
    for col in pivot_df.columns
]
pivot_df = pivot_df.reset_index()

print(pivot_df.head())
print(pivot_df.shape[0]) # Note we have 149 hotels instead of 150 - solution in next cell

**Verifying Missing Hotel Data**

After filtering and pivoting the data, we expect to have 150 hotels, but only 149 appear in our pivot table. This indicates that one (or more) of the top 150 hotels has no records for the top 40 check-in dates used in our analysis.

The code above does the following:
1. **Identify Missing Hotels:**  
   It compares the complete list of top 150 hotels (`top_150_hotels`) with the hotel names present in the pivoted DataFrame (`pivot_df`). Any hotel that is not present is added to the `missing_hotels` list.

2. **Check Data for Each Missing Hotel:**  
   For each missing hotel, it filters `filtered_checkin_df` (which already contains only records from the top 40 check-in dates) to see if there are any records for that hotel.  
   - If the resulting DataFrame is empty, it confirms that the hotel indeed has no data for those check-in dates.  
   - This explains why the hotel did not appear in the pivot table.

By verifying that the missing hotel has no records in the filtered data, we can conclude that the drop in the number of hotels is due to the absence of data for those check-in dates rather than an error in our processing pipeline.



In [ ]:
# Assuming you have already defined:
# - top_150_hotels: the complete list of top 150 hotel names.
# - pivot_df: the pivoted DataFrame after grouping and filtering.
# - filtered_checkin_df: the DataFrame filtered by top 40 check-in dates.
#
# And the missing hotels are identified as:
missing_hotels = [hotel for hotel in top_150_hotels if hotel not in pivot_df['Hotel Name'].values]
print("Missing hotels:", missing_hotels)

# For each missing hotel, check if there is any record in the filtered_checkin_df.
for hotel in missing_hotels:
    hotel_records = filtered_checkin_df[filtered_checkin_df['Hotel Name'] == hotel]
    print(f"\nRecords for missing hotel '{hotel}':")
    print(hotel_records)  # This should print an empty DataFrame if no data is present.



**Task**

For each hotel, we have a 160-dimensional vector of discount prices (one for each combination of Checkin Date and Discount Code). The goal is to normalize these prices so that, for each hotel, the lowest valid discount price becomes 0 and the highest becomes 100. Any missing value (indicated by `-1`) should remain unchanged.

**Plan**

1. **Define a Normalization Function:**  
   Create a function (`normalize_row`) that:
   - Filters out the missing values (`-1`) from the row.
   - Computes the minimum and maximum values among the valid discount prices.
   - Applies the normalization formula:
     $$
     \text{normalized\_price} = \frac{(\text{price} - \text{min\_price})}{(\text{max\_price} - \text{min\_price})} \times 100
     $$
   - Handles the case where all valid prices are equal (to avoid division by zero) by setting them to 0.

2. **Apply the Function Row-wise:**  
   Normalize the discount prices for each hotel (i.e., for each row) by applying the function to all columns except the "Hotel Name".

3. **Round and Convert to Integers:**  
   After normalization, round the values to the nearest integer and convert them to an integer type, ensuring that the normalized prices are stored as integers.



In [ ]:
def normalize_row(row):
    valid_mask = row != -1
    valid_prices = row[valid_mask]
    
    if valid_prices.empty:
        return row
    
    min_price = valid_prices.min()
    max_price = valid_prices.max()
    
   # Avoid division by zero if all valid prices are identical
    if min_price == max_price:
        row[valid_mask] = 0
    else:
        # Compute the normalized values, round them, and cast to int
        normalized_values = ((row[valid_mask] - min_price) / (max_price - min_price)) * 100
        row[valid_mask] = normalized_values.round(0).astype(int)
    
    return row


pivot_df.iloc[:, 1:] = pivot_df.iloc[:, 1:].apply(normalize_row, axis=1)

for col in pivot_df.columns[1:]:
    pivot_df[col] = pd.to_numeric(pivot_df[col], errors='coerce')
    pivot_df[col] = pivot_df[col].astype("Int64")




print(pivot_df.head(5))



**Save to CSV**

In [ ]:
hotels_clustering_data = "./hotels_clustering_data.csv"
pivot_df.to_csv(hotels_clustering_data, index=False)

### Hierarchical Clustering

**Task**

Using the normalized discount prices for each hotel, we will perform hierarchical clustering to group hotels that exhibit similar pricing behaviors. We have a 160-dimensional feature vector for each hotel (each dimension corresponds to a specific (Checkin Date, Discount Code) pair).

**Plan**

1. **Prepare the Data:**  
   - Load the saved CSV file (`hotels_clustering_data.csv`).
   - Separate the "Hotel Name" column (for labeling) from the numeric feature columns.

2. **Perform Hierarchical Clustering:**  
   - Use SciPy's `linkage` function with Ward's method (which works well with Euclidean distance) to compute the clustering.
   - Generate a linkage matrix that represents the hierarchical clustering.

3. **Plot the Dendrogram:**  
   - Use SciPy's `dendrogram` function to visualize the hierarchical clustering.
   - Label each leaf in the dendrogram with the corresponding hotel name to help interpret the clusters.


In [ ]:
!pip install plotly

In [ ]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage

def create_dendrogram_from_csv(csv_path, color_threshold=825, width=1200, height=1800):

    clu_df = pd.read_csv(csv_path)
    hotel_names = clu_df["Hotel Name"].values
    X = clu_df.drop("Hotel Name", axis=1).values
    Z = linkage(X, method='ward')

    fig = ff.create_dendrogram(
        X,
        orientation='left',
        labels=hotel_names,
        color_threshold=color_threshold,
        linkagefun=lambda x: Z
    )

    fig.update_layout(
        width=width,
        height=height
    )
    fig.show()
    
    return fig

color_threshold_list = [825, 750, 625, 500]
for color_threshold in color_threshold_list:
    print (f'color_threshold={color_threshold}')
    create_dendrogram_from_csv("hotels_clustering_data.csv", color_threshold=color_threshold)


### Results analysis

We performed hierarchical clustering on a dataset of hotels, where each hotel is represented by a **160-dimensional vector** of normalized discount prices. In simpler terms, each hotel’s vector shows *how* it discounts (and by how much) across different dates and discount codes. The dendrogram below clusters these hotels based on their similarity in discounting patterns.

Below, we examine **five different “cut” distances**—825, 750, 625, 500, and an additional view with fewer, broader clusters—and describe the cluster/subgroup formations you see in each figure.

---

#### Overall Explanation of the Dendrogram
- **X-axis**: The distance (or dissimilarity) at which clusters merge. Larger values mean more dissimilar groups.  
- **Y-axis**: The list of hotels, labeled along the left side.  
- **Colored Branches**: Each color indicates a cluster or subgroup under the specified distance threshold.

In general:  
- Hotels that **merge at smaller distances** (farther to the left in the dendrogram) are quite similar in how they price their discounts.  
- If you follow the dendrogram to the right until a major branch merges, that indicates hotels (or clusters of hotels) that are more dissimilar in their pricing behavior.

---

#### 1. Cut at Distance ≈ 825

![Cut at ~825](./images/clustering-825.png)  

- We see **three main clusters** forming at this high distance threshold.  
- **Group 1** (green), **Group 2** (red), and **Group 3** (blue) represent broad differences in discounting strategies. 
- Within each group, hotels share overall similarities in their discount patterns, but we’re not yet seeing the finer differences.

---

#### 2. Cut at Distance ≈ 750

![Cut at ~750](./images/clustering-750.png)  

- Lowering the threshold to **~750** begins to reveal **subgroups** within **Groups 1** and **3**, while **Group 2** remains mostly intact.  
- **Group 1** (previously green) starts splitting into multiple smaller clusters. These subgroups suggest that, while these hotels share a broad discount pattern, some differences in exact pricing behavior are now visible.  
- **Group 3** also subdivides more than before.
- **Group 2** is still one cluster, which indicates that these hotels are, as a whole, more cohesive or less varied in their discount strategies at this distance.

---

#### 3. Cut at Distance ≈ 625

![Cut at ~625](./images/clustering-625.png)  

- By **~625**, **Group 1** and **Group 3** break into several distinct subgroups, illustrating more granular differences.  
- **Group 2** is still a single cluster—indicating even at this more fine-grained level, the hotels in Group 2 remain very similar.  
- This suggests **Group 2** has a stable, uniform discount pattern, or at least they differ less from each other than from the other groups.

---

#### 4. Cut at Distance ≈ 500

![Cut at ~500](./images/clustering-500.png)  

- Finally, at **~500**, **Group 2** splits into **two subgroups**, showing that there are *at least* two distinct discounting patterns within Group 2 when we look closely.  
- **Groups 1 and 3** now fragment into many smaller subgroups. If you count them, there could be several mini-clusters in each.  
- Each **subgroup** here represents hotels that are very closely aligned in how they handle discounts across time—likely direct competitors or hotels following near-identical pricing guidelines.

---

#### Meaning of Subgroups

1. **Competitive Landscape**  
   - Hotels in the **same subgroup** are likely *direct competitors*, as they share nearly identical discount structures and timelines.

2. **Revenue Management Strategy**  
   - Subgroups often align with brand or chain policies (e.g., a single chain might appear as a tight cluster if they use centralized pricing software).  
   - The level of granularity around **500** distance reveals the *very fine details* of each hotel’s promotional activities.

3. **Marketing & Differentiation**  
   - If you manage a hotel in these clusters, you might examine how close neighbors in the dendrogram are discounting.  
   - Being in a tight cluster might motivate you to **differentiate** or **align** your pricing further.

4. **Insights for Group 2**  
   - Group 2’s stability up to a lower threshold (625) suggests a coherent pricing approach among its hotels. They only start splitting at ~500, which indicates that their differences are more subtle and only become apparent under a finer lens.

---

#### Putting It All Together
At **higher thresholds** (825+), we see broad strokes: *three main pricing behaviors*.  As we lower the cut distance, we discover **finer distinctions** within each major group:
- **Groups 1 & 3** develop several distinct subgroups earlier (around 750 or 625).  
- **Group 2** remains consistent longer and only splits around 500, revealing that even seemingly uniform clusters can exhibit small internal differences if you zoom in far enough.

In essence, **each subgroup** helps us understand small sets of hotels that might share very close discounting patterns—key information for competitive analysis and strategic pricing decisions.



### diving even deeper - (checking the hotel stars and average price with the pricing strategy)

The results from the last section were interesting, so i decided to run the same clustering but with the hotels stars, prices and discounts to see if some patterns emerge.

The new label now contains:
- stars
- avg price
- avg discount
- avg discount rate 

in this format:

`(stars) - price - discount - discount rate`

example:

(5) - 3898 - 3663 - 6% 

In [ ]:
import pandas as pd

pivot_df = pd.read_csv("hotels_clustering_data.csv")
df = pd.read_csv("./hotels_data_changed.csv")

pivot_df["Hotel Name"] = pivot_df["Hotel Name"].astype(str).str.strip()
hotel_counts = df["Hotel Name"].value_counts()
top_150_hotels = hotel_counts.head(150).index

summary_df = (
    df[df["Hotel Name"].isin(top_150_hotels)]
    .groupby("Hotel Name")
    .agg({"Original Price": "mean", "Discount Price": "mean", "Hotel Stars": "first"})
    .reset_index()
)

summary_df["Hotel Name"] = summary_df["Hotel Name"].astype(str).str.strip()
summary_df["Original Price"] = summary_df["Original Price"].round(0).astype(int)
summary_df["Discount Price"] = summary_df["Discount Price"].round(0).astype(int)
merged_df = pivot_df.merge(summary_df, on="Hotel Name", how="left")

merged_df["DiscountPerc"] = (((merged_df["Original Price"] - merged_df["Discount Price"]) / merged_df["Original Price"]) * 100).round(0).astype(int)
merged_df["Label"] = merged_df.apply(lambda row: f"({row['Hotel Stars']}) - {row['Original Price']} - {row['Discount Price']} - {row['DiscountPerc']}%", axis=1)

print(merged_df.head())
merged_df.to_csv("./hotels_clustering_data_with_summary.csv", index=False)


In [ ]:
import pandas as pd
import plotly.figure_factory as ff
from scipy.cluster.hierarchy import linkage

def create_dendrogram_from_csv(csv_path, color_threshold=825, width=1200, height=1800):
    clu_df = pd.read_csv(csv_path)
    if "Label" in clu_df.columns:
        labels = clu_df["Label"].values
        non_clustering = ["Hotel Name", "Label", "Hotel Stars", "Original Price", "Discount Price", "DiscountPerc"]
    else:
        labels = clu_df["Hotel Name"].values
        non_clustering = ["Hotel Name", "Hotel Stars", "Original Price", "Discount Price", "DiscountPerc"]
    X = clu_df.drop(columns=non_clustering, errors='ignore').values
    Z = linkage(X, method='ward')
    fig = ff.create_dendrogram(
        X,
        orientation='left',
        labels=labels,
        color_threshold=color_threshold,
        linkagefun=lambda x: Z
    )
    fig.update_layout(width=width, height=height)
    fig.show()
    return fig

color_threshold_list = [825, 750, 625, 500]
for color_threshold in color_threshold_list:
    print(f'color_threshold={color_threshold}')
    create_dendrogram_from_csv("hotels_clustering_data_with_summary.csv", color_threshold=color_threshold)


#### Hierarchical Clustering with Star Rating, Avg Price, and Discounts

We re-ran clustering with four features:
1. **Hotel star rating**  
2. **Avg nightly price**  
3. **Avg absolute discount**  
4. **Avg discount rate (%)**

Each dendrogram label is `(stars) – price – discount – discount rate`.

#### 1. Main Observations

- **Star Rating & Price** dominate the first major splits:
  - **Budget/Midscale** (2–3★, ~1,200–1,800 price) form one big group.
  - **Upscale/Luxury** (4–5★, 2,500–4,000+) cluster separately.
- **Discount Patterns** refine clusters within each star tier:
  - Hotels sharing similar price but different discount behavior split into distinct subgroups.
  - Aggressive vs. conservative discounters separate at lower distance thresholds.

#### 2. Cluster Insights

1. **Large Distance Clusters**:
   - Budget vs. Luxury hotels split clearly by base price and star rating.
   - Some **mixed star** groups appear if their price/discount overlap.

2. **Subgroups at Tighter Distances**:
   - Within the same star tier, variations in discount size or percentage form separate sub-clusters.
   - Outliers can include a high-star hotel with heavy discounts or a low-star with high pricing.

#### 3. Takeaways

- **Star Rating is Primary**; discount strategies refine subgroups.  
- Hotels in tight clusters likely share near-identical pricing/discount policies, suggesting **direct competition**.  
- Examine outliers for unique pricing approaches or potential repositioning strategies.

## step 4 - Building a Predictive Model for Hotel Pricing Dynamics Using Snapshot Data



## step 5 - PySpark & Mllib for step 3

### Installation

Spark requires a java version of 8 or 11 to work

In [1]:
%%capture 
!conda install -c conda-forge openjdk=11 -y

config spark to work with our env java version(11)

In [5]:
import os 
java_home_path = os.popen('dirname $(dirname $(which java))').read().strip()
os.environ["JAVA_HOME"] = java_home_path
print(f"JAVA_HOME is set to: {os.environ['JAVA_HOME']}")


JAVA_HOME is set to: /opt/anaconda3/envs/DS-101-Final


In [ ]:
%%capture 
!pip install pyspark plotly

### Create Spark session

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, lit, concat, first, array, udf
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import BisectingKMeans
from pyspark.sql.functions import min as spark_min

try:
    spark.stop()
except:
    pass

spark = SparkSession.builder.appName("HotelClustering").getOrCreate()
print("Create Spark session") 

25/02/08 11:08:23 WARN Utils: Your hostname, Yoavs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.120 instead (on interface en0)
25/02/08 11:08:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/08 11:08:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Create Spark session


### Data Loading and Filtering

- Load the CSV file.
- Select the top 150 hotels (by number of records).
- Select the top 40 checkin dates.

In [6]:
file_path = "./hotels_data_changed.csv"

df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)

top150_hotels = df.groupBy("Hotel Name").agg(count("*").alias("cnt")) .orderBy(col("cnt").desc()).limit(150)


df_top150 = df.join(top150_hotels.select("Hotel Name"), on="Hotel Name", how="inner")


top40_dates = df_top150.groupBy("Checkin Date").agg(count("*").alias("cnt")).orderBy(col("cnt").desc()).limit(40)

df_top150_dates = df_top150.join(top40_dates.select("Checkin Date"), on="Checkin Date", how="inner")

df_top150_dates.head()


Row(Checkin Date=datetime.date(2015, 8, 13), Hotel Name='The Peninsula New York', Snapshot ID=1, Snapshot Date=datetime.date(2015, 7, 17), Days=5, Original Price=4370, Discount Price=4240, Discount Code=1, Available Rooms=3, Hotel Stars=5, DayDiff=27, WeekDay='Thursday', DiscountDiff=130, DiscountPerc=2.9748283752860414)

### Convert to 160-dims vector

In [8]:
df_grouped = df_top150_dates.groupBy("Hotel Name", "Checkin Date", "Discount Code").agg(spark_min("Discount Price").alias("minDiscountPrice"))

df_grouped = df_grouped.withColumn("date_code", concat(col("Checkin Date"), lit("_"), col("Discount Code")))

df_pivot = df_grouped.groupBy("Hotel Name").pivot("date_code").agg(first("minDiscountPrice"))

df_pivot = df_pivot.fillna(-1)

df_pivot.head(3)

25/02/08 11:13:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(Hotel Name='Bentley Hotel', 2015-08-12_1=-1, 2015-08-12_2=-1, 2015-08-12_3=-1, 2015-08-12_4=-1, 2015-08-13_1=-1, 2015-08-13_2=-1, 2015-08-13_3=-1, 2015-08-13_4=-1, 2015-08-19_1=-1, 2015-08-19_2=-1, 2015-08-19_3=-1, 2015-08-19_4=-1, 2015-08-26_1=-1, 2015-08-26_2=-1, 2015-08-26_3=-1, 2015-08-26_4=-1, 2015-08-27_1=-1, 2015-08-27_2=-1, 2015-08-27_3=980, 2015-08-27_4=-1, 2015-08-28_1=-1, 2015-08-28_2=-1, 2015-08-28_3=-1, 2015-08-28_4=-1, 2015-09-09_1=-1, 2015-09-09_2=-1, 2015-09-09_3=-1, 2015-09-09_4=-1, 2015-09-10_1=-1, 2015-09-10_2=-1, 2015-09-10_3=-1, 2015-09-10_4=-1, 2015-09-11_1=-1, 2015-09-11_2=-1, 2015-09-11_3=-1, 2015-09-11_4=-1, 2015-09-16_1=-1, 2015-09-16_2=-1, 2015-09-16_3=-1, 2015-09-16_4=-1, 2015-09-17_1=-1, 2015-09-17_2=-1, 2015-09-17_3=-1, 2015-09-17_4=-1, 2015-09-18_1=-1, 2015-09-18_2=-1, 2015-09-18_3=-1, 2015-09-18_4=-1, 2015-09-30_1=-1, 2015-09-30_2=-1, 2015-09-30_3=-1, 2015-09-30_4=-1, 2015-10-01_1=-1, 2015-10-01_2=-1, 2015-10-01_3=-1, 2015-10-01_4=-1, 2015-10-02_1=-

We might find ourself in a situation where some of the hotels don't have a column for all the 160 dates + discount codes.
So we would ensure thy all have that.

In [10]:
top40_list = [row["Checkin Date"] for row in top40_dates.collect()]
discount_codes = [1, 2, 3, 4]

# Build the expected column names (format: "YYYY-MM-DD_1", etc.)
expected_cols = [f"{date}_{code}" for date in top40_list for code in discount_codes]

# Add any missing expected columns with default -1
existing_cols = df_pivot.columns
for col_name in expected_cols:
    if col_name not in existing_cols:
        df_pivot = df_pivot.withColumn(col_name, lit(-1))

# Reorder the DataFrame columns so that they appear in the desired order:
df_pivot = df_pivot.select(["Hotel Name"] + expected_cols)
df_pivot.head(3)

[Row(Hotel Name='Bentley Hotel', 2015-11-11_1=-1, 2015-11-11_2=-1, 2015-11-11_3=-1, 2015-11-11_4=-1, 2015-10-14_1=-1, 2015-10-14_2=-1, 2015-10-14_3=-1, 2015-10-14_4=-1, 2015-11-04_1=-1, 2015-11-04_2=-1, 2015-11-04_3=-1, 2015-11-04_4=-1, 2015-08-19_1=-1, 2015-08-19_2=-1, 2015-08-19_3=-1, 2015-08-19_4=-1, 2015-10-28_1=-1, 2015-10-28_2=-1, 2015-10-28_3=-1, 2015-10-28_4=-1, 2015-10-21_1=1405, 2015-10-21_2=1403, 2015-10-21_3=1394, 2015-10-21_4=1389, 2015-11-06_1=-1, 2015-11-06_2=-1, 2015-11-06_3=-1, 2015-11-06_4=-1, 2015-08-12_1=-1, 2015-08-12_2=-1, 2015-08-12_3=-1, 2015-08-12_4=-1, 2015-11-05_1=1184, 2015-11-05_2=1138, 2015-11-05_3=1133, 2015-11-05_4=1179, 2015-10-22_1=-1, 2015-10-22_2=-1, 2015-10-22_3=-1, 2015-10-22_4=-1, 2015-11-12_1=-1, 2015-11-12_2=-1, 2015-11-12_3=-1, 2015-11-12_4=-1, 2015-09-10_1=-1, 2015-09-10_2=-1, 2015-09-10_3=-1, 2015-09-10_4=-1, 2015-10-29_1=-1, 2015-10-29_2=-1, 2015-10-29_3=2076, 2015-10-29_4=-1, 2015-09-09_1=-1, 2015-09-09_2=-1, 2015-09-09_3=-1, 2015-09-09_4=-

### Normalization and Save to CSV

Normalize the 160 price columns row-by-row (scaling valid prices to a 0–100 range, leaving missing values as -1).

In [11]:
# Combine the 160 price columns into an array column
df_pivot = df_pivot.withColumn("prices_array", array(*expected_cols))

# Define a UDF for normalizing the prices for each hotel (ignoring -1 values)
def normalize_prices(prices):
    # Filter out missing values (-1)
    valid_prices = [p for p in prices if p != -1]
    if not valid_prices:
        return prices
    min_price = min(valid_prices)
    max_price = max(valid_prices)
    if min_price == max_price:
        return [0 if p != -1 else -1 for p in prices]
    normalized = []
    for p in prices:
        if p == -1:
            normalized.append(-1)
        else:
            norm_val = round(((p - min_price) / (max_price - min_price)) * 100)
            normalized.append(int(norm_val))
    return normalized

normalize_udf = udf(normalize_prices, ArrayType(IntegerType()))

# Apply the normalization UDF to create a new column with normalized prices.
df_pivot = df_pivot.withColumn("norm_prices_array", normalize_udf("prices_array"))

# Replace the original price columns with the normalized values.
for i, col_name in enumerate(expected_cols):
    df_pivot = df_pivot.withColumn(col_name, col("norm_prices_array")[i])

# Optionally, drop helper columns.
df_final = df_pivot.drop("prices_array", "norm_prices_array")


**Save to CSV**

In [12]:
pyspark_hotels_clustering_data= "./pyspark_hotels_clustering_data.csv"
df_final.write.option("header", "true").mode("overwrite").csv(pyspark_hotels_clustering_data)


### Clustering

#### Read CSV and assemble features

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Define the path to the CSV folder that was saved in Step 5.
pyspark_hotels_clustering_data = "./pyspark_hotels_clustering_data.csv"

# Read the CSV data (Spark will read all part files in the folder)
df_loaded = spark.read.option("header", "true").option("inferSchema", "true").csv(pyspark_hotels_clustering_data)

# Identify the feature columns (all columns except "Hotel Name")
feature_cols = [col for col in df_loaded.columns if col != "Hotel Name"]

# Assemble the 160 normalized price columns into a single feature vector.

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
df_features = assembler.transform(df_loaded)


root
 |-- Hotel Name: string (nullable = true)
 |-- 2015-11-11_1: integer (nullable = true)
 |-- 2015-11-11_2: integer (nullable = true)
 |-- 2015-11-11_3: integer (nullable = true)
 |-- 2015-11-11_4: integer (nullable = true)
 |-- 2015-10-14_1: integer (nullable = true)
 |-- 2015-10-14_2: integer (nullable = true)
 |-- 2015-10-14_3: integer (nullable = true)
 |-- 2015-10-14_4: integer (nullable = true)
 |-- 2015-11-04_1: integer (nullable = true)
 |-- 2015-11-04_2: integer (nullable = true)
 |-- 2015-11-04_3: integer (nullable = true)
 |-- 2015-11-04_4: integer (nullable = true)
 |-- 2015-08-19_1: integer (nullable = true)
 |-- 2015-08-19_2: integer (nullable = true)
 |-- 2015-08-19_3: integer (nullable = true)
 |-- 2015-08-19_4: integer (nullable = true)
 |-- 2015-10-28_1: integer (nullable = true)
 |-- 2015-10-28_2: integer (nullable = true)
 |-- 2015-10-28_3: integer (nullable = true)
 |-- 2015-10-28_4: integer (nullable = true)
 |-- 2015-10-21_1: integer (nullable = true)
 |-- 201

DataFrame[Hotel Name: string, 2015-11-11_1: int, 2015-11-11_2: int, 2015-11-11_3: int, 2015-11-11_4: int, 2015-10-14_1: int, 2015-10-14_2: int, 2015-10-14_3: int, 2015-10-14_4: int, 2015-11-04_1: int, 2015-11-04_2: int, 2015-11-04_3: int, 2015-11-04_4: int, 2015-08-19_1: int, 2015-08-19_2: int, 2015-08-19_3: int, 2015-08-19_4: int, 2015-10-28_1: int, 2015-10-28_2: int, 2015-10-28_3: int, 2015-10-28_4: int, 2015-10-21_1: int, 2015-10-21_2: int, 2015-10-21_3: int, 2015-10-21_4: int, 2015-11-06_1: int, 2015-11-06_2: int, 2015-11-06_3: int, 2015-11-06_4: int, 2015-08-12_1: int, 2015-08-12_2: int, 2015-08-12_3: int, 2015-08-12_4: int, 2015-11-05_1: int, 2015-11-05_2: int, 2015-11-05_3: int, 2015-11-05_4: int, 2015-10-22_1: int, 2015-10-22_2: int, 2015-10-22_3: int, 2015-10-22_4: int, 2015-11-12_1: int, 2015-11-12_2: int, 2015-11-12_3: int, 2015-11-12_4: int, 2015-09-10_1: int, 2015-09-10_2: int, 2015-09-10_3: int, 2015-09-10_4: int, 2015-10-29_1: int, 2015-10-29_2: int, 2015-10-29_3: int, 2

#### Clustering with MLlib (BisectingKMeans)

In [15]:
from pyspark.ml.clustering import BisectingKMeans

# Set the number of clusters (adjust k as needed)
bkmeans = BisectingKMeans(featuresCol="features", predictionCol="cluster", k=4)

# Train the model
model = bkmeans.fit(df_features)

# Add the cluster assignments to the DataFrame
df_clustered = model.transform(df_features)

# Show the hotel names along with their cluster assignments
df_clustered.select("Hotel Name", "cluster").show(truncate=False)


25/02/08 11:39:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


+-----------------------------------------------------+-------+
|Hotel Name                                           |cluster|
+-----------------------------------------------------+-------+
|Bentley Hotel                                        |0      |
|Westin New York at Times Square                      |2      |
|The Westin New York Grand Central                    |2      |
|Super 8 Brooklyn   Park Slope Hotel                  |0      |
|Four Seasons Hotel New York                          |3      |
|Omni Berkshire Place                                 |2      |
|DoubleTree by Hilton Metropolitan - New York City    |3      |
|Dumont NYC-an Affinia hotel                          |1      |
|Hampton Inn Manhattan Downtown-Financial District    |2      |
|Eventi Hotel a Kimpton Hotel                         |2      |
|Magnuson Convention Center Hotel                     |3      |
|Courtyard Newark Elizabeth                           |1      |
|Park Hyatt New York                    

#### Visualization with PCA and Plotly

Since the features are 160-dimensional, we use PCA to reduce them to 2 dimensions for visualization. Then, we convert the Spark DataFrame to a Pandas DataFrame and use Plotly Express to create a scatter plot.

In [16]:
from pyspark.ml.feature import PCA
pca = PCA(k=2, inputCol="features", outputCol="pcaFeatures")
pca_model = pca.fit(df_clustered)
df_pca = pca_model.transform(df_clustered)

pandas_df = df_pca.select("Hotel Name", "cluster", "pcaFeatures").toPandas()

# Split the PCA features into two separate columns for plotting
pandas_df["pca1"] = pandas_df["pcaFeatures"].apply(lambda x: x[0])
pandas_df["pca2"] = pandas_df["pcaFeatures"].apply(lambda x: x[1])


25/02/08 11:42:38 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [17]:
import plotly.express as px

fig = px.scatter(
    pandas_df,
    x="pca1",
    y="pca2",
    color="cluster",
    hover_data=["Hotel Name"],
    title="Hotel Clusters Visualization (PCA Reduced)"
)
fig.show()


#### Stop the spark session

In [18]:
spark.stop()